In [ ]:
!pip install cantera


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 9.8 MB/s eta 0:00:00


In [ ]:
pip install --upgrade cantera


In [ ]:
import cantera as ct
import matplotlib.pyplot as plt

import subprocess

# Replace 'gri30.cti' with the name of your Chemkin mechanism file
mechanism_file = 'gri30.cti'
yaml_file = 'gri30.yaml'

result = subprocess.run(['cti2yaml', mechanism_file, '-o', yaml_file], check=False, capture_output=True)

if result.returncode != 0:
    print(f"Error: {result.stderr.decode()}")


# Convert Chemkin mechanism to YAML format

yaml_file = ct.convert_chemical_file('gri30.cti', fmt='yaml', outName='gri30.yaml')

# Create a Cantera Solution object

gas = ct.Solution.from_yaml(yaml_file)

# Define the mechanism and initial state

mechanism=ct.mechanism('gri30.cti')

# Define the state for contstant-volume combustion

T = 800 # Initial Temperature (K)
P = 1000000 # Initial Pressure (Pa)

# Define the fuel and oxidizer compositions

fuel = 'n-heptane'
oxidizer = 'O2:N2:H2O=4:1.5:0.3'

# Set the initial state and create the reactor object

state = ct.State(mechanism, T, P)
state.TPX = {fuel:1.0,oxidizer:1.0}

# Define the reactor object

r = ct.State(mechanism, T, P)

# Set hydrogen energy share (HES) values

hes_values = [0, 1, 5, 10, 25, 50]

# Define engien load values

load_values = [0.25, 0.5, 0.75, 1.0]

# Initialize data storage arrays

power_data = []
hrr_data = []
efficiency_data = []
CO_emissions_data = []

# Loop through HES values

for hes in hes_values:
  # Modify fuel composition based on HES
  fuel_mol = ct.Quantity(fuel,1.0-hes/100)
  hydrogen_mol = ct.Quantity('H2',hes/100)
  fuel_mixture = {fuel_mol:1.0,hydrogen_mol:1.0}

  # Update reactor state with modified fuel composition

  state.TPX = fuel_mixture

  # Loop through engine load values

  for load in load_values:
    # Set the equivalence ratio
    equivalence_ratio = load/0.75
    # Perform constant volume combustion simulation
    r.run(equivalence_ratio)
    # Calculate engine power
    power = r.work()
    # Calculate heat release rate
    hrr = r.thermo.heat_release_rate()
    # Calculate brake thermal efficiency
    efficiency = power/(r.thermo.enthalpy_inlet()-r.thermo.enthalpy_outlet)
    # Calculate CO emmisions
    CO_emissions = r.thermo.mole_fractions('CO')
    # Store results in data arrays
    power_data.append([hes, load, power])
    hrr_data.append([hes, load, efficiency])
    CO_emissions_data.append([hes, load, CO_emissions])

    # Process and plot the results

for data_type, data_array in zip(['power', 'hrr', 'efficiency', 'CO_emissions'], [power_data, hrr_data, efficiency_data, CO_emissions_data]):
    # Process data into 2D arrays for each load
    processed_data = []
    for load in load_values:
        filtered_data = [data[1:] for data in data_array if data[0] == hes and data[1] == load]
        processed_data.append(filtered_data[0])

# Create subplots
fig, ax = plt.subplots()

# Plot data for each load
for load_index, load_data in enumerate(processed_data):
    load_label = 'Load:' + str(load_values[load_index] * 100) + '%'
    ax.plot(hes_values, [data[2] for data in load_data], label=load_label)

# Set plot labels and title
ax.set_xlabel('Hydrogen Energy Share (%)')
ax.set_ylabel(data_type)
ax.set_title('Effect of Hydrogen Share ' + data_type)
ax.legend()

# Show plot
plt.show()



Error: usage: cti2yaml [-h] [--input-encoding] [--quiet] [--no-validate] input [output]
cti2yaml: error: unrecognized arguments: -o gri30.yaml



AttributeError: ignored

In [ ]:
import cantera as ct
import matplotlib.pyplot as plt
import subprocess

# Replace 'gri30.cti' with the name of your Chemkin mechanism file
mechanism_file = 'gri30.cti'
yaml_file = 'gri30.yaml'

# Run the cti2yaml command using subprocess
subprocess.run(['cti2yaml', mechanism_file, '-o', yaml_file], check=True)

# Create a Cantera Solution object
gas = ct.Solution.from_yaml(yaml_file)

# Define the state for constant-volume combustion
T = 800  # Initial Temperature (K)
P = 1000000  # Initial Pressure (Pa)

# Define the fuel and oxidizer compositions
fuel = 'n-heptane'
oxidizer = 'O2:N2:H2O=4:1.5:0.3'

# Set the initial state and create the reactor object
state = ct.State(gas, T, P)
state.TPX = {fuel: 1.0, oxidizer: 1.0}

# Define engine load values
load_values = [0.25, 0.5, 0.75, 1.0]

# Initialize data storage arrays
power_data = []
hrr_data = []
efficiency_data = []
CO_emissions_data = []

# Loop through engine load values
for load in load_values:
    # Set the equivalence ratio
    equivalence_ratio = load / 0.75
    # Perform constant volume combustion simulation
    gas.TPX = T, P, {fuel: 1.0, oxidizer: equivalence_ratio}
    r = ct.IdealGasConstPressureReactor(gas)
    sim = ct.ReactorNet([r])
    sim.advance_to_steady_state()
    # Calculate engine power
    power = r.thermo.work()
    # Calculate heat release rate
    hrr = r.thermo.heat_release_rate()
    # Calculate brake thermal efficiency
    efficiency = power / (r.thermo.enthalpy_mass - gas.h())
    # Calculate CO emissions
    CO_emissions = r.thermo['CO'].Y[0]
    # Store results in data arrays
    power_data.append([load, power])
    hrr_data.append([load, hrr])
    efficiency_data.append([load, efficiency])
    CO_emissions_data.append([load, CO_emissions])

# Process and plot the results
fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs = axs.flatten()
data_types = ['Power', 'Heat Release Rate', 'Efficiency', 'CO Emissions']

for i, (data_type, data_array) in enumerate(zip(data_types, [power_data, hrr_data, efficiency_data, CO_emissions_data])):
    axs[i].plot([data[0] for data in data_array], [data[1] for data in data_array], marker='o')
    axs[i].set_xlabel('Engine Load')
    axs[i].set_ylabel(data_type)
    axs[i].set_title(f'{data_type} vs. Engine Load')

plt.tight_layout()
plt.show()

CalledProcessError: ignored